# Notebook 2: Exploratory_Data_Analysis_Demographic
## Introdução
Neste notebook, vamos carregar os dados da tabela `Demographic` que foram salvos em formato Parquet no notebook anterior e realizar uma análise exploratória dos dados, com o objectivo de melhor compreender os dados e testar as transformações a realizar ao parquet_raw.

## Passos

1. **Carregar Dados do Parquet**
    - Carregar os dados do arquivo Parquet `demo_raw`.

2. **Mostrar o Schema e as Primeiras Linhas dos Dados**
    - Verificar o schema dos dados carregados.
    - Exibir as primeiras linhas do DataFrame.

3. **Análise Exploratória dos Dados**
    - Contar o número de linhas no DataFrame.
    - Realizar estatísticas descritivas.
    - Verificar valores nulos.
    - Validar colunas.
    - Elaborar Schema

5. **Conclusão**
    - Resumir as descobertas iniciais e preparar os dados para análises futuras.


# 1. Carregar Dados do Parquet

In [0]:
# Carregar dados do Parquet
df_demo = spark.read.parquet('dbfs:/FileStore/FAERS-grupo-4/demo_raw')

# 2. Mostrar o Schema e as Primeiras Linhas dos Dados

In [0]:
df_demo.printSchema()

In [0]:
df_demo.limit(50).display()

# 3. Análise Exploratória dos Dados

## 1. Analise de Numero de Registos e Verificação de Colunas com Valores Null

In [0]:
df_demo.count()

In [0]:
df_demo.describe().display()

Primeiras impressões sobre a tabela e o seu schema

|-- primaryid: long (nullable = true) - identificador do relatório não pode ser null.  
 |-- caseid: integer (nullable = true) - identificador do caso não pode ser null.  
 |-- caseversion: integer (nullable = true) - identificador da versão do caso não pode ser null.  
 |-- i_f_code: string (nullable = true) - Codigo que representa se o registo é um follow-up ou o primeiro.  
 |-- event_dt: integer (nullable = true) - Data do inicio da reação adversa, nem todos os relatorios tem essa info (Pode ser null).  
 |-- mfr_dt: integer (nullable = true) - Data em que o fabricante foi notificado pela primeira vez.  
 |-- init_fda_dt: integer (nullable = true) - Data em que a FDA recebeu a primeira versão deste caso.  
 |-- fda_dt: integer (nullable = true) - Data em que recebeu o caso, nos follow-ups aparece com mfr_dt, não pode ser null.  
 |-- rept_cod: string (nullable = true) - Codigo para o tipo reporte representa o tempo que demorou a ser enviado e pode-se tirar info de quem enviou o relatorio aqui. Não tem nulls.  
 |-- auth_num: string (nullable = true) - Numero do caso da entidade regulatoria quando disponível. Pode ter nulls.  
 |-- mfr_num: string (nullable = true) - Codigo do relatório para o fabricante. Pode ter nulls.  
 |-- mfr_sndr: string (nullable = true) - Codigo do fabricante que enviou o relatorio, quando não encontrado tem o nome(verbatim) da organização que enviou. Não pode ter nulls.  
 |-- lit_ref: string (nullable = true) - Referencia literária quando disponível, tem os ultimos 500 caracteres quando excede 500 caracters. Pode ter nulls.  
 |-- age: integer (nullable = true) - Idade em numero. Pode ter nulls.  
 |-- age_cod: string (nullable = true) - Codigo da unidade usada para idade. Pode ser null.   
 |-- age_grp: string (nullable = true) - Codigo para o grupo de etario. Pode ser null.  
 |-- sex: string (nullable = true) - Codigo para o sexo. Pode ser null.  
 |-- e_sub: string (nullable = true) - Y/N para se foi submetido electronicamente. Deveria ser booleano. Não pode ser null.  
 |-- wt: double (nullable = true) - valor do peso. Pode ser null  
 |-- wt_cod: string (nullable = true) - Codigo para a unidade de peso. Pode ser null mas deverá ter o mesmo numero de registos que o wt.  
 |-- rept_dt: integer (nullable = true) - Data de quando o relatorio foi enviado. Não pode ter nulls.  
 |-- to_mfr: string (nullable = true) - Y/N caso seja um relatorio voluntario. Pode ter nulls.  
 |-- occp_cod: string (nullable = true) - Codigo da ocupação (relativa a este contexto) de quem reportou.  
 |-- reporter_country: string (nullable = true) - Codigo do pais que reportou o caso.  
 |-- occr_country: string (nullable = true) - Codigo do pais onde ocorreu o caso.  


## 2. Analise por colunas

### 1. Coluna i_f_cod

Vamos manter as colunas primaryid, caseid e caseversion. Vamos avançar para perceber se a coluna i_f_cod segue as regras definidas.

In [0]:
df_demo.groupBy(df_demo['i_f_code']).count().display()

Confirma-se não há dados que fujam ao definido. Vamos alterar esta coluna para boleano em que true é quando é follow-up.

In [0]:
from pyspark.sql.functions import when
df_demo = df_demo.withColumn("folup_rpt", when(df_demo['i_f_code'] == 'F', True).otherwise(False))

Validar se a conversão foi bem feita

In [0]:
df_demo.where(df_demo['i_f_code']=='F').display()

In [0]:
df_demo.where(df_demo['i_f_code']=='I').display()

Conversão bem realizada. Vamos eliminar a coluna original

In [0]:
df_demo=df_demo.drop(df_demo['i_f_code'])

In [0]:
df_demo.limit(1000).display()

### 2. Colunas do tipo data (event_dt, mfr_dt, init_fda_dt, fda_dt, rept_dt)

Vamos agora avançar para tratar as colunas com datas, começando com event_dt.

Sabemos através das notas da documentação:  
- As datas podem estar no formato YYYYMMDD ou YYYYMM ou YYYY.  
- YYYYMM é igual a YYYYMM01.  
- YYYY é igual a YYYY0101. 

In [0]:
from pyspark.sql.functions import to_date, when, length, col, expr
def convert_date(df,date_col,dest_col):
  # Suponha que o nome da sua coluna seja 'data_numero'
  df = df.withColumn("data_string", col(date_col).cast("string"))

  df = df.withColumn(dest_col, 
      when(length(col("data_string")) == 8, to_date(col("data_string"), "yyyyMMdd"))
      .when(length(col("data_string")) == 6, to_date(expr("concat(data_string, '01')"), "yyyyMMdd"))
      .when(length(col("data_string")) == 4, to_date(expr("concat(data_string, '0101')"), "yyyyMMdd"))
  )
  return df.drop(col("data_string"))

In [0]:
convert_date(df_demo,'event_dt','event_date').select('event_dt','event_date').limit(1000).display()

In [0]:
#Validar a não perda de registos:
convert_date(df_demo,'event_dt','event_date').select('event_date').where(col('event_date').isNotNull()).count()

In [0]:
df_demo.select('event_dt').where(col('event_dt').isNotNull()).count()

Verifica-se a perda de dois registos em relação ao count do describe 964780. Vamos validar...

Vamos tentar validar se algum registo não segue o padrão que definimos na função

In [0]:
df_test = df_demo.withColumn("data_string", col('event_dt').cast("string"))
df_test.select('data_string').where( (length(col("data_string")) != 6)).where( (length(col("data_string")) != 4)).where( (length(col("data_string")) != 8)).display()

Como não é encontrado nenhum significado nestes valores vamos optar por ignorar e aplicar a transformação.
Verificou-se que efectivamente esta solução funciona e que pode ser aplicada às restantes colunas

In [0]:
df_demo = convert_date(df_demo,'event_dt','event_dt')
df_demo.display()

Vamos agora validar a existencia de valores que não correspondem aos padrões nas outras colunas

In [0]:
# colunas com date:
colunas_dt = [x for x in df_demo.columns if x.endswith('_dt')]
colunas_dt

In [0]:
# event_dt já está tratado por isso
colunas_dt = colunas_dt[1:]
colunas_dt

In [0]:
for coluna in colunas_dt:
    df_test = df_demo.withColumn("data_string", col(coluna).cast("string"))
    df_test.select('data_string').where( (length(col("data_string")) != 6)).where( (length(col("data_string")) != 4)).where( (length(col("data_string")) != 8)).display()

In [0]:
df_demo.select(colunas_dt).limit(10).display()

Verfica-se que só é um problema da event_dt e que se pode aplicar nas restantes colunas

In [0]:
for coluna in colunas_dt:
    df_demo = convert_date(df_demo,coluna,coluna)
df_demo.display()

### 3. Coluna rept_cod

In [0]:
# Criar DataFrame de mapeamento de códigos para descrição das fontes
mapping_data = [
    ("EXP", "Expedited (15-Day)"),
    ("PER", "Periodic (Non-Expedited)"),
    ("DIR", "Direct"),
    ("5DAY", "5-Day"),
    ("30DAY", "30-Day")
]
mapping_columns = ["rept_cod", "rept_typ_txt"]
df_mapping = spark.createDataFrame(mapping_data, mapping_columns)

# Join entre df_rept_cod e o DataFrame de mapeamento
df_demo_test = df_demo.join(df_mapping, on="rept_cod", how="left")

# Mostre o DataFrame resultante
df_demo_test.select('rept_cod','rept_typ_txt').display()

Parece estar a funcionar vamos ver se existem nulls na coluna criada

In [0]:
df_demo_test.select('rept_cod','rept_typ_txt').where(col('rept_typ_txt').isNull()).where(col('rept_cod').isNotNull()).display()

Confirma-se que o mapeamento do 'rept_cod' foi bem executado, resultando na ausência nulls. 


In [0]:
# Criar DataFrame de mapeamento de códigos para descrição das fontes
mapping_data = [
    ("EXP", "Expedited (15-Day)"),
    ("PER", "Periodic (Non-Expedited)"),
    ("DIR", "Direct"),
    ("5DAY", "5-Day"),
    ("30DAY", "30-Day")
]
mapping_columns = ["rept_cod", "rept_typ_txt"]
df_mapping = spark.createDataFrame(mapping_data, mapping_columns)

# Join entre df_rept_cod e o DataFrame de mapeamento
df_demo= df_demo.join(df_mapping, on="rept_cod", how="left")

# Mostre o DataFrame resultante
df_demo.select('rept_cod','rept_typ_txt').display()

### 4. Coluna auth_num

Avançamos no estudo da coluna 'auth_num':

In [0]:
df_demo.select('auth_num').distinct().limit(1000).display()

In [0]:
df_demo.select('auth_num').where(col('auth_num').isNull()).count()

Numa rápida análise concluimos que existem vários formatos de 'auth_num' não havendo na documentação referência de uma nomenclatura especifica, nesse sentido não vamos aplicar nenhuma transformação nesta coluna. Nem sempre fará sentido estar esta coluna preenchida uma vez que nem sempre foi reportado por uma autoridade reguladora. 

### 5. Coluna mfr_num

Agora procedemos a validação da coluna 'mfr_num'


In [0]:
df_demo.select('mfr_num').display()

In [0]:
df_demo.select('mfr_num').where(col('mfr_num').isNull()).count()

In [0]:
#proporção percentual nulls 'mfr_cod'
87382/2157280 * 100

Nem sempre está disponivel o identificador do relatório interno do fabricante, mas concluimos que esta coluna não necessita de tranformação. 

### 6. Coluna mfr_sndr

In [0]:
df_demo.select('mfr_sndr').distinct().display()

In [0]:
df_demo.select('mfr_sndr').where(col('mfr_sndr').isNull()).count()

Esta coluna não precisa de qualquer tratamento, estando sempre disponivel qual o fabricante. Ora por identificador, ora por verbatim.

### 7. Coluna lit_ref

In [0]:
df_demo.select('lit_ref').display()

In [0]:
lit_ref_perc_null=(df_demo.select('lit_ref').where(col('lit_ref').isNull()).count())/(df_demo.count())*100
print(f'A percentagem de nulls na coluna de referencias literárias é :{lit_ref_perc_null}')

Apesar da elevada percentagem de valores a Null (92.86%), não vamos fazer a eliminação da tabela, uma vez que para avaliações futuras ao dataset poderá ser util ter toda a informação disponivel. 

### 8. Colunas age

Vamos agora as colunas relacionadas com a idade

In [0]:
# colunas com age:
colunas_age = [x for x in df_demo.columns if x.startswith('age')]
colunas_age

In [0]:
df_demo.select(colunas_age).display()

In [0]:
df_demo.select('age_cod').distinct().display()

Vamos testar normalizar as idades em anos

In [0]:
from pyspark.sql.functions import round
df_demo_age_normalized = df_demo.withColumn(
    "age_in_years",
      when(col("age_cod") == "MON", round(col("age") / 12, 2))
      .when(col("age_cod") == "HR", round(col("age")/(24*365) , 2))
      .when(col("age_cod") == "DY", round(col("age")/365 , 2))
      .when(col("age_cod") == "DEC", round(col("age")*10 , 2))
      .when(col("age_cod") == "WK", round(col("age")/52 , 2))
      .when(col("age_cod") == "YR", col("age")).otherwise(None)
    )

In [0]:
df_demo_age_normalized.display()

In [0]:
df_demo.select('age').where(col('age').isNotNull()).count()

In [0]:
df_demo.select('age_cod').where(col('age_cod').isNotNull()).count()

In [0]:
df_demo_age_normalized.select('age','age_cod','age_in_years','age_grp').where(col('age').isNotNull()).where(col('age_cod').isNull()).display()

A normalização funcionou mas existem registos que não se verifica a existencia de age_cod. Vamos tentar corrigir alguns registos com o uso do age_grp, assim


In [0]:
df_demo_age_normalized = df_demo_age_normalized.withColumn(
    "age_cod",
    when(
        (col("age_grp") == "A") & (col("age_cod").isNull()) & (col("age").isNotNull()) & (col("age")>18) & (col("age")<150),
        "YR"
    )
    .when(
        (col("age_grp") == "E") & (col("age_cod").isNull()) & (col("age").isNotNull()) & (col("age")>18) & (col("age")<150),
        "YR"
    )
    .when( col("age_cod").isNotNull(), 
        col("age_cod")

    )  # Defina o valor padrão para outras condições
)
df_demo_age_normalized.select('age','age_cod','age_in_years','age_grp').where(col('age').isNotNull()).where(col('age_cod').isNull()).display()

A transformção feita corrige a falta de age_cod para years nos casos possiveis (150<age<18 e age_grp==A|E).  
Vamos aplicar as transformações no df_demo, primeiro corrigindo a falta de age_cod e depois criando a coluna age_in_years  

In [0]:
df_demo = df_demo.withColumn(
    "age_cod",
    when(
        (col("age_grp") == "A") & (col("age_cod").isNull()) & (col("age").isNotNull()) & (col("age")>18) & (col("age")<150),
        "YR"
    )
    .when(
        (col("age_grp") == "E") & (col("age_cod").isNull()) & (col("age").isNotNull()) & (col("age")>18) & (col("age")<150),
        "YR"
    )
    .when( col("age_cod").isNotNull(), 
        col("age_cod")

    )  # Defina o valor padrão para outras condições
)
df_demo.select('age','age_cod','age_grp').where(col('age').isNotNull()).where(col('age_cod').isNull()).display()

In [0]:
from pyspark.sql.functions import round
df_demo = df_demo.withColumn(
    "age_in_years",
      when(col("age_cod") == "MON", round(col("age") / 12, 2))
      .when(col("age_cod") == "HR", round(col("age")/(24*365) , 2))
      .when(col("age_cod") == "DY", round(col("age")/365 , 2))
      .when(col("age_cod") == "DEC", round(col("age")*10 , 2))
      .when(col("age_cod") == "WK", round(col("age")/52 , 2))
      .when(col("age_cod") == "YR", col("age")).otherwise(None)
    )
df_demo.select('age','age_cod','age_in_years','age_grp').display()

Vamos agora tentar preencher os valores em falta da coluna age_grp usando a classificação da FDA  
Neonates - from birth through the first 28 days of life  
Infants - 29 days to less than 2 years  
Children - 2 years to less than 12 years  
Adolescents - aged 12 through 21  
Adults - 21 to less than 65 years
https://www.fda.gov/medical-devices/products-and-medical-procedures/pediatric-medical-devices

In [0]:

df_test = df_demo.withColumn("age_grp",
   when(col("age_grp").isNotNull(),col("age_grp")) 
    .when(col("age_in_years") < (28.0/365), "N")
    .when((col("age_in_years") >= (28.0/365)) & (col("age_in_years") < 2), "I")
    .when((col("age_in_years") >= 2) & (col("age_in_years") < 12), "C")
    .when((col("age_in_years") >= 12) & (col("age_in_years") < 22), "T")
    .when((col("age_in_years") >= 22) & (col("age_in_years") < 65), "A")
    .when(col("age_in_years") >= 65 , "E")
    
)
df_test.where(col('age_grp').isNull()).count()


In [0]:
df_demo.where(col('age_grp').isNull()).count()

A transformação resultou menos nulls na coluna (1606931-861456=745475)  
Vamos então aplicar a trasnformação

In [0]:
df_demo = df_demo.withColumn("age_grp",
   when(col("age_grp").isNotNull(),col("age_grp")) 
    .when(col("age_in_years") < (28.0/365), "N")
    .when((col("age_in_years") >= (28.0/365)) & (col("age_in_years") < 2), "I")
    .when((col("age_in_years") >= 2) & (col("age_in_years") < 12), "C")
    .when((col("age_in_years") >= 12) & (col("age_in_years") < 22), "T")
    .when((col("age_in_years") >= 22) & (col("age_in_years") < 65), "A")
    .when(col("age_in_years") >= 65 , "E")
    
)
df_demo.select(colunas_age).display()

In [0]:
df_demo.select(colunas_age+['age_in_years']).display()

### 9. Coluna sex

concluido tratamento das colunas idade, vamos avançar para coluna sex

In [0]:
df_demo.select('sex').distinct().display()

Verificamos a existencia de nulls, no contexto dado na documentação existe a classificação de unknown (UNK). Vamos corrigir então.

In [0]:
df_demo.groupBy('sex').count().display()

In [0]:
df_demo.fillna({'sex':'UNK'}).groupBy('sex').count().display()

A transformação funcionou por isso vamos aplicar

In [0]:
df_demo = df_demo.fillna({'sex':'UNK'})
df_demo.select('sex').display()

### 10. Coluna e_sub

Vamos agora tratar da coluna e_sub

In [0]:
df_demo.groupBy('e_sub').count().display()

Os valores estão uniformizados e como é uma coluna de sim ou não passará para booleano

In [0]:
df_demo.withColumn("e_sub",when(col("e_sub")=='Y',True).otherwise(False)).groupBy('e_sub').count().display()

Transformação bem sucedida. Vamos aplicar.

In [0]:
df_demo = df_demo.withColumn("e_sub",when(col("e_sub")=='Y',True).otherwise(False))
df_demo.printSchema()

### 11. Colunas wt

Vamos avançar para as colunas wt e wt_cod 

In [0]:
df_demo.groupBy('wt_cod').count().display()

In [0]:
df_demo.where(col('wt').isNotNull()).groupBy('wt_cod').count().display()

In [0]:
df_demo.select(['wt','wt_cod']).where((col('wt').isNotNull()) & (col('wt_cod')=='UNK')).display()

Segundo a documentação UNK não é um valor valido vamos substituir por null. Por outro lado vamos conseguir normalizar praticamente todos o pesos disponíveis porque apenas 2 são UNK nas unidades.

In [0]:
df_demo.withColumn("wt_cod", when(col("wt_cod") == "UNK", None).otherwise(col("wt_cod"))).groupBy('wt_cod').count().display()

Vamos aplicar a trasformação.

In [0]:
df_demo = df_demo.withColumn("wt_cod", when(col("wt_cod") == "UNK", None).otherwise(col("wt_cod")))

Vamos agora normalizar os pesos em kgs

In [0]:
from pyspark.sql.functions import round
df_demo.withColumn(
    "wt_in_kg",
      when(col("wt_cod") == "KG", round(col("wt"), 3))
      .when(col("wt_cod") == "LBS", round(col("wt")/2.2046 , 3))
      .when(col("wt_cod") == "GMS", round(col("wt")/1000 , 3)).otherwise(None)
    ).select(['wt','wt_cod','wt_in_kg']).where(col('wt_cod') =='LBS').display()

Verifica-se uma boa conversão. vamos aplicar.

In [0]:
df_demo = df_demo.withColumn(
    "wt_in_kg",
      when(col("wt_cod") == "KG", round(col("wt"), 3))
      .when(col("wt_cod") == "LBS", round(col("wt")/2.2046 , 3))
      .when(col("wt_cod") == "GMS", round(col("wt")/1000 , 3)).otherwise(None)
    )

### 12. Coluna to_mfr

avançamos agora para a coluna to_mfr

In [0]:
df_demo.groupBy('to_mfr').count().display()

Segundo a documentação UNK não é o valor valido por isso vamos substituir por null e transformar a coluna em booleano uma vez que é outro campo sim ou não

In [0]:
df_demo.withColumn("to_mfr", when(col("to_mfr") == "UNK", None).otherwise(col("to_mfr"))).groupBy('to_mfr').count().display()

In [0]:
df_demo.withColumn("to_mfr",when(col("to_mfr")=='Y',True).when(col("to_mfr")=='N',False).otherwise(None)).groupBy('to_mfr').count().display()

Vamos agora aplicar a ultima que transforma tudo como queremos

In [0]:
df_demo = df_demo.withColumn("to_mfr",when(col("to_mfr")=='Y',True).when(col("to_mfr")=='N',False).otherwise(None))

### 13. Coluna occp_cod

Vamos agora tratar a coluna occp_cod

In [0]:
df_demo.groupBy('occp_cod').count().display()

Mais uma vez UNK não é um valor presente na documentação por isso será substituido por null

In [0]:
df_demo.withColumn("occp_cod", when(col("occp_cod") == "UNK", None).otherwise(col("occp_cod"))).groupBy('occp_cod').count().display()

Após validação vamos aplicar

In [0]:
df_demo = df_demo.withColumn("occp_cod", when(col("occp_cod") == "UNK", None).otherwise(col("occp_cod")))

Vamos agora criar uma coluna occp_text para ser mais facil a leitura

In [0]:
# Criar DataFrame de mapeamento de códigos para descrição das fontes
mapping_data = [
    ("MD", "Physician"),
    ("PH", "Pharmacist"),
    ("OT", "Other health-professional"),
    ("LW", "Lawyer"),
    ("CN", "Consumer")
]
mapping_columns = ["occp_cod", "occp_txt"]
df_mapping = spark.createDataFrame(mapping_data, mapping_columns)

# Join entre df_rept_cod e o DataFrame de mapeamento
df_demo_test = df_demo.join(df_mapping, on="occp_cod", how="left")
df_demo_test.select(['occp_cod','occp_txt']).distinct().display()


Criação da coluna resulta. vamos criar então para o df_demo

In [0]:
mapping_data = [
    ("MD", "Physician"),
    ("PH", "Pharmacist"),
    ("OT", "Other health-professional"),
    ("LW", "Lawyer"),
    ("CN", "Consumer")
]
mapping_columns = ["occp_cod", "occp_txt"]
df_mapping = spark.createDataFrame(mapping_data, mapping_columns)

# Join entre df_rept_cod e o DataFrame de mapeamento
df_demo = df_demo.join(df_mapping, on="occp_cod", how="left")

### 14. Colunas country

Vamos agora analisar as colunas relativas aos países


In [0]:
df_demo.groupBy('reporter_country').count().display()
df_demo.groupBy('occr_country').count().display()

In [0]:
df_demo.limit(1000).display()


In [0]:
df_countries = spark.read.csv("dbfs:/FileStore/NCIt_GENC_Terminology.txt", header=True, inferSchema=True, sep='\t')
df_countries.display()

In [0]:
df_countries.select(['GENC 2 Letter Code','GENC Name (FDA Standard)']).dropna().display()

In [0]:
df_countries = df_countries.select(['GENC 2 Letter Code','GENC Name (FDA Standard)']).dropna().withColumnsRenamed({'GENC 2 Letter Code':'reporter_country','GENC Name (FDA Standard)':'reporter_country_text'})


In [0]:
df_countries.display()

In [0]:
df_demo_test = df_demo.join(df_countries, on="reporter_country", how="left")

In [0]:
df_demo_test.select(['reporter_country','reporter_country_text']).distinct().limit(1000).display()

os nulls e UNKs desaparecem por isso temos de adicionar no df_countries. Segundo a documentação os nulls devem ser 'Country not specified', por isso vamos substituir no df_demo os nulls e UNKs por 'Country not specified' e adicionar esse valor ao df_countries.

In [0]:
df_demo.withColumn("reporter_country", when(length(col("reporter_country")) != 2, 'Country not specified').otherwise(col("reporter_country"))).fillna('Country not specified').select(["reporter_country"]).distinct().display()

Funciona, vamos aplicar nas duas colunas de countries a substituição

In [0]:
df_demo = df_demo.withColumn("reporter_country", when(length(col("reporter_country")) != 2, 'Country not specified').otherwise(col("reporter_country"))).fillna({'reporter_country': 'Country not specified'})
df_demo = df_demo.withColumn("occr_country", when(length(col("occr_country")) != 2, 'Country not specified').otherwise(col("occr_country"))).fillna({'occr_country': 'Country not specified'})

df_demo.groupBy('reporter_country').count().display()
df_demo.groupBy('occr_country').count().display()

agora para criar as colunas de texto ainda precisamos de adicionar a linha 'Country not specified' ao df_countries

In [0]:
new_rows = [('Country not specified', 'Country not specified')]
new_df = spark.createDataFrame(new_rows, ['reporter_country','reporter_country_text'])
df_countries = df_countries.union(new_df)

In [0]:
df_countries.display()

resumo de como criar o df_countries para criar a coluna reporter_country_text. Para fazer na occr_country é necessário mudar os nomes das colunas

In [0]:
df_countries = spark.read.csv("dbfs:/FileStore/NCIt_GENC_Terminology.txt", header=True, inferSchema=True, sep='\t').select(['GENC 2 Letter Code','GENC Name (FDA Standard)']).dropna().withColumnsRenamed({'GENC 2 Letter Code':'reporter_country','GENC Name (FDA Standard)':'reporter_country_text'})
new_rows = [('Country not specified', 'Country not specified')]
new_df = spark.createDataFrame(new_rows, ['reporter_country','reporter_country_text'])
df_countries = df_countries.union(new_df)
df_countries.display()

In [0]:
# teste com reporter country
df_demo.join(df_countries, on="reporter_country", how="left").select(["reporter_country","reporter_country_text"]).distinct().display()

Vamos aplicar então

In [0]:
df_demo = df_demo.join(df_countries, on="reporter_country", how="left")

Avançando para a occr_country

In [0]:
df_demo.join(df_countries.withColumnsRenamed({'reporter_country':'occr_country','reporter_country_text':'occr_country_text'}), on="occr_country", how="left").select(['occr_country','occr_country_text']).distinct().display()

Funciona. Vamos aplicar

In [0]:
df_demo = df_demo.join(df_countries.withColumnsRenamed({'reporter_country':'occr_country','reporter_country_text':'occr_country_text'}), on="occr_country", how="left")

Transformações concluidas

In [0]:
df_demo.display(1000)

Vamos agora então fazer o schema


## 3. Elaboração do Schema de Leitura

In [0]:
from pyspark.sql.types import *
schema_demo = StructType ([
  StructField('primaryid',LongType(),False,{'description':'Unique number for identifying a FAERS report'}),
  StructField('caseid',LongType(),False,{'description':'Number for identifying a FAERS case.'}),
  StructField('caseversion',IntegerType(),False,{'description':'Safety Report Version Number'}),
  StructField('i_f_code',StringType(),False,{'description':'Code for initial or follow-up status of report, as reported by manufacturer.'}),
  StructField('event_dt',IntegerType(),True,{'description':'Date the adverse event occurred or began'}),
  StructField('mfr_dt',IntegerType(),True,{'description':'Date manufacturer first received initial information'}),
  StructField('init_fda_dt',IntegerType(),True,{'description':'Date FDA received first version (Initial) of Case'}),
  StructField('fda_dt',IntegerType(),True,{'description':'Date FDA received Case'}),
  StructField('rept_cod',StringType(),False,{'description':'Code for the type of report submitted'}), 
  StructField('auth_num',StringType(),True,{'description':'Regulatory Authority’s case report number'}),
  StructField('mfr_num',StringType(),True,{'description':'Manufacturer’s unique report identifier'}),
  StructField('mfr_sndr',StringType(),False,{'description':'Coded name of manufacturer sending report'}),
  StructField('lit_ref',StringType(),True,{'description':'Literature Reference information'}),
  StructField('age',IntegerType(),True,{'description':'Numeric value of patient’s age at event.'}),
  StructField('age_cod',StringType(),True,{'description':'Unit abbreviation for patient’s age'}),
  StructField('age_grp',StringType(),True,{'description':'Patient Age Group code'}),
  StructField('sex',StringType(),True,{'description':'Code for patient’s sex'}),
  StructField('e_sub',StringType(),False,{'description':'Whether this report was submitted under the electronic submissions procedure for manufacturers'}),
  StructField('wt',DoubleType(),True,{'description':'Numeric value of patient’s weight.'}),
  StructField('wt_cod',StringType(),True,{'description':'Unit abbreviation for patient’s weight'}),
  StructField('rept_dt',IntegerType(),True,{'description':'Date report was sent'}),
  StructField('to_mfr',StringType(),True,{'description':'Whether voluntary reporter also notified manufacturer'}),
  StructField('occp_cod',StringType(),True,{'description':'Abbreviation for the reporter’s type of occupation in the latest version of a case.'}),
  StructField('reporter_country',StringType(),True, {'description': "Reporter Country Code"}),
  StructField('occr_country',StringType(),True,{'description':'The country where the event occurred.'}),
  StructField('CORRUPT_DATA',StringType(),True)
  ]
)
df_demo = spark.read.format('parquet').schema(schema_demo).load('dbfs:/FileStore/FAERS-grupo-4/demo_raw',mode="PERMISSIVE",columnNameOfCorruptRecord="CORRUPT_DATA")
df_demo.display()
df_demo.select('CORRUPT_DATA').distinct().display()
df_demo.printSchema()
fields = df_demo.schema.fields
["column_name= %s type= %s metadata= %s" % (t.name, t.dataType, t.metadata) for t in fields]

# 4. Conclusões

Transformações a fazer no pipeline:   
- load do schema inicial com metadados   
- i_f_code para bool.  
- aplcar transformação para data de todas as colunas '_dt' através da função convert_date.  
- Fazer mapeamento da coluna 'rept_cod' para 'rept_typ_txt'   
- corrgir nulls na coluna age_cod  
- Normalizar 'age' em 'years'  
- corrigir nulls na coluna age_grp  
- Fazer fill dos nulls do sex por UNK  
- fazer conversão para booleano da coluna e_sub  
- fazer a substituição de UNK por  null na coluna wt_cod  
- fazer a conversão da coluna to_mfr para booleano substituindo UNK por nulls  
- fazer a substituição de UNK por  null na coluna occp_cod  
- criar a coluna occp_txt  
- substituir UNKs e nulls por Country not specified  
- criar df_countries  
- Criar coluna reporter_country_text  
- Alterar titulos df_countries para occr_country e criar occr_country_text
